!git clone https://www.modelscope.cn/datasets/BJQW14B/bs_challenge_financial_14b_dataset.git

# sqlite3

In [ ]:
import sqlite3

# 创建连接
conn = sqlite3.connect('dataset/博金杯比赛数据.db')

cursor = conn.cursor()

# # 创建表
# cursor.execute('''CREATE TABLE yourTableName
#                   (column1 datatype, column2 datatype)''')

# # 插入数据
# cursor.execute("INSERT INTO yourTableName VALUES (value1, value2)")

# # 提交改动
# conn.commit()

# 查询数据
cursor.execute("SELECT * FROM 基金基本信息 limit 1")
results = cursor.fetchall()
for row in results:
    print(row)

cursor.execute("SELECT * FROM A股公司行业划分表 limit 1")
results = cursor.fetchall()
for row in results:
    print(row)
cursor.close()
conn.close()

In [ ]:
import jsonlines

def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content


def write_jsonl(path, content):
    with jsonlines.open(path, "w") as json_file:
        json_file.write_all(content)


content = read_jsonl("/Users/jxy/Projects/challenge/bs_challenge_financial_14b_dataset/question.json")


In [ ]:
for cont in content:
    cont['answer'] = "unknown"
write_jsonl("/Users/jxy/Projects/challenge/bs_challenge_financial_14b_dataset/submit_result.jsonl", content=content)

# try pypdf

In [167]:
import re
pattern = r'(.*招股意向书)?[\s\n]+.*[\d-]+[\s\n]*'

from langchain.document_loaders import *

# pdffiles = [
#     "43571629ce9fb30b44ccefdca9f637102c16d1a2.PDF",
#     "91b4426b075560a1a45247f9cfa9fa73d56c945c.PDF",
#     "f587290218d881e18e88fc1431b022b2c5aca81a.PDF",
#     "f9e84ce0edd5279773b3ca1f36a9e39d6ceaf220.PDF",
#     "d336d607e1d431cbfe1f313e2234a13fcf49a16e.PDF",
#     "a6f8156c08a1096c46470a1c5e1229daaaedf06e.PDF",
#     "54d148902b889679830174597830f0d0f22c1073.PDF",
# ]
import os
pdfpath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/"
txtpath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/newtxt/"
filepaths = os.listdir(pdfpath)
filepaths.sort()
print("total file="  + str(len(filepaths)))

for idx, filename in enumerate(filepaths):
    print("begin i=" + str(idx) + " filename= " + filename)
    flagSub = False
    # first PyPDFLoader ， then PyMuPDFLoader
    with open(txtpath + filename.replace("PDF", "txt"), "w") as out:
        try:
            docs = PyPDFLoader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/" + filename).load()
        except Exception as e:
            print("error i=" + str(idx) + " filename= " + filename)
            docs = PyMuPDFLoader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/" + filename).load()
        for doc in docs:
            # only for head
            max_len = len(doc.page_content)
            head = ""
            rest = ""
            if doc.metadata['page'] != 0: # 忽略第一页
                if max_len > 150:
                    head = doc.page_content[0:150]
                    rest = doc.page_content[150:]
                else:
                    head = doc.page_content
                res_head = re.sub(pattern, "", head, flags=re.DOTALL)
                if len(res_head) != len(head):
                    flagSub = True
                out.write(res_head)
                out.write(rest)
            else:
                out.write(doc.page_content)
        print("end i =" + str(idx) + ", filename= " +filename + " flagSub=" + str(flagSub))
print("end")

total file=80


In [ ]:
from PyPDF2 import PdfReader
reader = PdfReader("/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/91b4426b075560a1a45247f9cfa9fa73d56c945c.PDF")
page = reader.pages[9]
print(page.extract_text((0,90)))

In [ ]:
# 蛮快的，并且质量还行，没有异常停止
import pdfplumber
docs = []
pdffiles = [
    "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/91b4426b075560a1a45247f9cfa9fa73d56c945c.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/f587290218d881e18e88fc1431b022b2c5aca81a.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/f9e84ce0edd5279773b3ca1f36a9e39d6ceaf220.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/d336d607e1d431cbfe1f313e2234a13fcf49a16e.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/a6f8156c08a1096c46470a1c5e1229daaaedf06e.PDF",
    # "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/54d148902b889679830174597830f0d0f22c1073.PDF",
]
for file_path in pdffiles:
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
        # Extract the text
        # first_page = pdf.pages[0]
            # print(page.extract_text())
            # print("\n\n")
        # Extract the data
            # print(page.extract_text_lines()) 
            tables = page.extract_table()
            if tables is not None:
                print("\ntext:\n")
                print(page.extract_text(layout=True))  
                print("\ntables:\n")
                for table in tables:
                    print(str(table).replace("\\n", ""))
                break;
        # # Extract the images
        # images = pdf.get_images()
        # for image in images:
        #     print(image["page_number"])
            # with open(f"image_{image['page_number']}.jpg", "wb") as f:
            #     f.write(image["data"])


In [ ]:
# PyMuPDF 质量一般
import sys, pathlib, fitz
for file_path in pdffiles:
    with fitz.open(file_path) as doc:  # open document
        # text = chr(12).join([page.get_text() for page in doc])
        page = doc[9]
        tables = page.find_tables()
        print(doc[9].get_text())
        print("\ntables:\n")
        for table in tables:
            print(str(table).replace("\\n", ""))
        # print(text[0:3000])
    # write as a binary file to support non-ASCII characters
    # pathlib.Path(fname + ".txt").write_bytes(text.encode())

In [ ]:
# bord质量还行，但是会抛异常
import typing
import sys, pathlib
from borb.pdf import Document
from borb.pdf import PDF
from borb.toolkit import SimpleTextExtraction
outpath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/newtxt/a.txt"
for file_path in pdffiles:
# read the Document
    doc: typing.Optional[Document] = None
    l: SimpleTextExtraction = SimpleTextExtraction()
    with open(file_path, "rb") as in_file_handle:
        doc = PDF.loads(in_file_handle, [l])

    # check whether we have read a Document
    # assert doc is not None

    # print the text on the first Page
print(l.get_text()[9])
# for page in l.get_text().pages:
#     print(page)
#     break;
    # pathlib.Path(outpath).write_bytes(page)

In [154]:
# 异常处理
from langchain.document_loaders import *
# PyPDFLoader PyPDFium2Loader PDFMinerLoader PyMuPDFLoader PDFPlumberLoader MathpixPDFLoader 
filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/43571629ce9fb30b44ccefdca9f637102c16d1a2.PDF"
loaders = [
    PyPDFLoader(filepath), # 失败
    PyPDFium2Loader(filepath), # 可以
    # PDFMinerLoader(filepath), # 失败
    PyMuPDFLoader(filepath), #  可以, 这个速度比较快，作为备选
    # PDFPlumberLoader(filepath),  # 失败
    # MathpixPDFLoader(filepath),
]
for loader in loaders:
    try:
        print(loader.__class__.__name__)
        docs = loader.load()
        page = docs[0]
        print(page)
    except Exception as e:
        print(e)
        print("error")

PyPDFLoader
Invalid Elementary Object starting with b'>' @1370: b'10011.4 1 Tf\n1150013>c9814>]TJ\n8.778.77140<0015003140623d20003>Tj\n10.785 3354203'
error
PyPDFium2Loader
page_content='天津力生制药股份有限公司\r\nTIANJIN LISHENG PHARMACEUTICAL CO., LTD \r\n（天津市南开区黄河道 491 号）\r\n首次公开发行股票\r\n招 股 意 向 书\r\n 保荐机构（主承销商）\r\n天津市南开区宾水西道 8 号\n' metadata={'source': '/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/43571629ce9fb30b44ccefdca9f637102c16d1a2.PDF', 'page': 0}
PDFMinerLoader


Data-loss while decompressing corrupted data


'int' object is not iterable
error
PyMuPDFLoader
page_content=' \n天津力生制药股份有限公司 \nTIANJIN LISHENG PHARMACEUTICAL CO., LTD \n（天津市南开区黄河道 491 号） \n首次公开发行股票 \n招 股 意 向 书 \n \n \n \n \n \n \n \n  保荐机构（主承销商） \n \n \n天津市南开区宾水西道 8 号 \n \n \n' metadata={'source': '/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/43571629ce9fb30b44ccefdca9f637102c16d1a2.PDF', 'file_path': '/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/43571629ce9fb30b44ccefdca9f637102c16d1a2.PDF', 'page': 0, 'total_pages': 395, 'format': 'PDF 1.4', 'title': '<4D6963726F736F667420576F7264202D203031D5D0B9C9D2E2CFF2CAE9A3A8D7EED0C2A3A9A3A834D4C231C8D5BBF1B5C3C5FACEC4A3A92E646F63>', 'author': 'user', 'subject': '', 'keywords': '', 'creator': 'PScript5.dll Version 5.2', 'producer': 'Acrobat Distiller 7.0 (Windows)', 'creationDate': "D:20100401095732+08'00'", 'modDate': "D:20100401095732+08'00'", 'trapped': ''}
PDFPlumberLoader


Data-loss while decompressing corrupted data


'int' object is not iterable
error


In [163]:
# 表格处理
from langchain.document_loaders import *
# PyPDFLoader PyPDFium2Loader PDFMinerLoader PyMuPDFLoader PDFPlumberLoader MathpixPDFLoader 
filepath = "/Users/jxy/Projects/person/ChallengeAI/2023bojin/data/pdf/91b4426b075560a1a45247f9cfa9fa73d56c945c.PDF"
loaders = [
    PyPDFLoader(filepath), # 比较好 \n
    PyPDFium2Loader(filepath), # 一般, \r\r
    # PDFMinerLoader(filepath), # 只有一页
    PyMuPDFLoader(filepath), # 一般 \n
    # PDFPlumberLoader(filepath),  # table 处理比较乱
    # MathpixPDFLoader(filepath), # 需要其他环境依赖
]
for loader in loaders:
    docs = loader.load();
    print(docs[9])


page_content='广州中海达卫星导航技术股份有限公司                                                      招股意向书  \n10 \n 第一节   释    义 \n在本招股意向书中，除非文意另有所指，下列简称具有以下特定意义：  \n一、普通词语  \n发行人、本公司、公司、\n股份公司、中海达、中海达股份、母公司  指 广州中海达卫星导航技术股份有限公司  \n原创数码、有限公司  指 广州原创数码科技有限公司，发行人整体变更前存续公司  \n控股股东、实际控制人  指 本公司第一大股东廖定海，持有本公司 1,505.5814 万股，占发\n行前股本总额的 40.1488% ，为本公司控股股东、实际控制人\n中海达测绘  指 广州市中海达测绘仪器有限公司，本公司全资子公司  \n华星定位  指 广州华星定位技术有限公司，本公司全资子公司  \n国信弘盛  指 国信弘盛投资有限公司  \nTrimble、天宝  指 Trimble Navigation Limited, 全球知名 GPS产品和解决方案提\n供商， NASDAQ 上市公司  \nLeica、徕卡  指 Leica Geosystems A.G. ，全球知名的测量仪器制造商、总部位\n于瑞士  \nTopcon、拓普康  指 Topcon Corporation ，日本测量仪器和医疗及工业用仪器制造\n商，东京交易所上市公司   \n南方测绘  指 广州南方测绘仪器有限公司  \n上海华测、华测  指 上海华测导航技术有限公司  \n合众思壮  指 北京合众思壮科技股份有限公司  \n苏州一光  指 苏州一光仪器有限公司  \nU-blox 指 U-blox Holding AG ，GPS芯片组供应商，瑞士交易所上市公\n司 \nSiRF 指 SiRF Technology, Inc. ，GPS芯片组供应商，总部位于美国  \nNovAtel、诺瓦泰  指 NovAtel Inc ，GPS产品和技术供应商，总部位于加拿大  \nJA V AD、佳瓦特  指 JA V AD GNSS Inc. ，GPS产品和技术供应商，总部位于美国  \nMagellan 、麦哲伦  指 Magellan Navigation Inc. ，GPS产品制造商